In [1]:
# %pip install -q -U bitsandbytes
# %pip install -q -U git+https://github.com/huggingface/transformers.git
# %pip install -q -U git+https://github.com/huggingface/peft.git
# %pip install -q -U git+https://github.com/huggingface/accelerate.git
# %pip install -q -U trl

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer

In [3]:
peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
)

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [5]:
# Load the entire model on the GPU 0
device_map = {"": 0}

In [6]:
from datasets import Dataset
import pandas as pd

# Load the data using pandas
data_file = "../data/ARCSolver_core_puzzles_40000.json"
df = pd.read_json(data_file)

# Convert the pandas dataframe to a dataset
dataset = Dataset.from_pandas(df)

def generate_prompt(data_point):
    text = '<s>[INST] ' + data_point["instruction"] + ' [/INST] ' + str(data_point["output"]) + '</s>'
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [7]:
# Load base model
model_name = "../merged_models/merged_model_step_2"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Load LLaMA tokenizer
base_model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
max_seq_length = 1250

new_adapter_name = "Finetuned_merge_002_v001"

output_dir = "../results/" + new_adapter_name

steps_per_epoch = len(dataset)//(per_device_train_batch_size*gradient_accumulation_steps)
print("Steps per epoch:", steps_per_epoch)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    max_steps=4000,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim="paged_adamw_8bit",
    save_strategy="steps",
    evaluation_strategy="no",
    save_steps=100,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True,
    warmup_steps=0.03,
    group_by_length=True,
    gradient_checkpointing=True,
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Steps per epoch: 10000


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [9]:
# Train model
trainer.train()

  0%|          | 0/4000 [00:00<?, ?it/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/shawn/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:467: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.2846, 'learning_rate': 0.00019995149963624728, 'epoch': 0.0}
{'loss': 0.1598, 'learning_rate': 0.00019990149926124448, 'epoch': 0.0}
{'loss': 0.3198, 'learning_rate': 0.00019990149926124448, 'epoch': 0.0}
{'loss': 0.2965, 'learning_rate': 0.00019990149926124448, 'epoch': 0.0}
{'loss': 0.3331, 'learning_rate': 0.00019985149888624167, 'epoch': 0.0}
{'loss': 0.2985, 'learning_rate': 0.00019980149851123885, 'epoch': 0.0}
{'loss': 0.1823, 'learning_rate': 0.00019975149813623603, 'epoch': 0.0}
{'loss': 0.235, 'learning_rate': 0.00019970149776123324, 'epoch': 0.0}
{'loss': 0.2388, 'learning_rate': 0.00019965149738623042, 'epoch': 0.0}
{'loss': 0.1966, 'learning_rate': 0.0001996014970112276, 'epoch': 0.0}
{'loss': 0.214, 'learning_rate': 0.00019955149663622478, 'epoch': 0.0}
{'loss': 0.2331, 'learning_rate': 0.000199501496261222, 'epoch': 0.0}
{'loss': 0.1791, 'learning_rate': 0.00019945149588621917, 'epoch': 0.0}
{'loss': 0.1652, 'learning_rate': 0.00019940149551121635, 'epoch': 0.

/home/shawn/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:467: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.1109, 'learning_rate': 0.00019515146363597728, 'epoch': 0.01}
{'loss': 0.1532, 'learning_rate': 0.00019510146326097446, 'epoch': 0.01}
{'loss': 0.101, 'learning_rate': 0.00019505146288597167, 'epoch': 0.01}
{'loss': 0.1583, 'learning_rate': 0.00019500146251096885, 'epoch': 0.01}
{'loss': 0.1059, 'learning_rate': 0.00019495146213596603, 'epoch': 0.01}
{'loss': 0.1259, 'learning_rate': 0.00019490146176096322, 'epoch': 0.01}
{'loss': 0.1133, 'learning_rate': 0.00019485146138596042, 'epoch': 0.01}
{'loss': 0.117, 'learning_rate': 0.0001948014610109576, 'epoch': 0.01}
{'loss': 0.1192, 'learning_rate': 0.0001947514606359548, 'epoch': 0.01}
{'loss': 0.1165, 'learning_rate': 0.00019470146026095197, 'epoch': 0.01}
{'loss': 0.1049, 'learning_rate': 0.00019465145988594918, 'epoch': 0.01}
{'loss': 0.0964, 'learning_rate': 0.00019460145951094636, 'epoch': 0.01}
{'loss': 0.1027, 'learning_rate': 0.00019455145913594354, 'epoch': 0.01}
{'loss': 0.1015, 'learning_rate': 0.0001945014587609407

...

/home/shawn/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:467: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0779, 'learning_rate': 4.5200339002542523e-05, 'epoch': 0.31}
{'loss': 0.1022, 'learning_rate': 4.515033862753971e-05, 'epoch': 0.31}
{'loss': 0.0794, 'learning_rate': 4.51003382525369e-05, 'epoch': 0.31}
{'loss': 0.1075, 'learning_rate': 4.505033787753409e-05, 'epoch': 0.31}
{'loss': 0.1139, 'learning_rate': 4.5000337502531276e-05, 'epoch': 0.31}
{'loss': 0.0678, 'learning_rate': 4.4950337127528464e-05, 'epoch': 0.31}
{'loss': 0.0801, 'learning_rate': 4.4900336752525645e-05, 'epoch': 0.31}
{'loss': 0.1319, 'learning_rate': 4.4850336377522834e-05, 'epoch': 0.31}
{'loss': 0.1128, 'learning_rate': 4.480033600252002e-05, 'epoch': 0.31}
{'loss': 0.1021, 'learning_rate': 4.475033562751721e-05, 'epoch': 0.31}
{'loss': 0.1298, 'learning_rate': 4.47003352525144e-05, 'epoch': 0.31}
{'loss': 0.0776, 'learning_rate': 4.4650334877511586e-05, 'epoch': 0.31}
{'loss': 0.117, 'learning_rate': 4.4600334502508774e-05, 'epoch': 0.31}
{'loss': 0.0836, 'learning_rate': 4.455033412750596e-05, 'ep

/home/shawn/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:467: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0815, 'learning_rate': 4.0200301502261276e-05, 'epoch': 0.32}
{'loss': 0.0948, 'learning_rate': 4.0150301127258464e-05, 'epoch': 0.32}
{'loss': 0.091, 'learning_rate': 4.0100300752255645e-05, 'epoch': 0.32}
{'loss': 0.1072, 'learning_rate': 4.0050300377252833e-05, 'epoch': 0.32}
{'loss': 0.0866, 'learning_rate': 4.000030000225002e-05, 'epoch': 0.32}
{'loss': 0.1546, 'learning_rate': 3.99502996272472e-05, 'epoch': 0.32}
{'loss': 0.0923, 'learning_rate': 3.990029925224439e-05, 'epoch': 0.32}
{'loss': 0.1094, 'learning_rate': 3.985029887724158e-05, 'epoch': 0.32}
{'loss': 0.1241, 'learning_rate': 3.980029850223877e-05, 'epoch': 0.32}
{'loss': 0.118, 'learning_rate': 3.9750298127235955e-05, 'epoch': 0.32}
{'loss': 0.0943, 'learning_rate': 3.9700297752233144e-05, 'epoch': 0.32}
{'loss': 0.1159, 'learning_rate': 3.965029737723033e-05, 'epoch': 0.32}
{'loss': 0.0898, 'learning_rate': 3.960029700222752e-05, 'epoch': 0.32}
{'loss': 0.122, 'learning_rate': 3.955029662722471e-05, 'epoc

KeyboardInterrupt: 

In [ ]:
# Save trained model
adapter_from_merged = "../adapters/" + new_adapter_name
trainer.model.save_pretrained(adapter_from_merged)